In [1]:
from wiki_dataset import WikiDataset
from embedding_generation import EmbeddingGenerator
from torch.utils.data import DataLoader

In [2]:
generator = EmbeddingGenerator(encoder='mpnet', version='v1', device='cuda')

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [3]:
wiki_dataset = WikiDataset(type='train', reduced=True)
wiki_dataloader = DataLoader(wiki_dataset, batch_size=500, num_workers=10, shuffle=False)

In [4]:
generator.generate(wiki_dataloader)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Block 0/26 done


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Block 1/26 done


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

...

Block 23/26 done


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Block 24/26 done


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Block 25/26 done
